In [15]:
from pkgman import include
include(["pandas", "polars", "rich", "tqdm", "requests", "matplotlib", "seaborn", "numpy", "igraph", "json", "pandarallel"])

pd = pandas
pl = polars
tqdm = tqdm
rq = requests
plt = matplotlib.pyplot
sns = seaborn
np = numpy
pp = rich.print
ig = igraph
pdl = pandarallel.pandarallel
pdl.initialize(progress_bar=True, nb_workers=8)

[pkgman] Installing and importing ['pandas', 'polars', 'rich', 'tqdm', 'requests', 'matplotlib', 'seaborn', 'numpy', 'igraph', 'json', 'pandarallel']...
[pkgman] 11 packages have been imported.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [16]:
df_path = "../data/result/users.all.jsonl"
df = pd.read_json(df_path, lines=True)
df.head(5)

,user_id,user_name,user_meta,team,follower_amount,follower_meta,following_amount,following_meta
0,ricardotalavera,Ricardo Talavera,"{""lastUserActivities"":[],""blogPosts"":[],""total...",None yet,0,[],0,[]
1,Teetouch,Teetouch Jaknamon,"{""lastUserActivities"":[],""blogPosts"":[],""total...",None,0,[],0,[]
2,crossdelenna,Cross,"{""lastUserActivities"":[{""time"":""2025-02-28T19:...",\n\t,0,[],1,[]
3,orestxherija,Orest Xherija,"{""lastUserActivities"":[],""blogPosts"":[],""total...",None yet,0,[],0,[]
4,nlptown,NLP Town,"{""lastUserActivities"":[],""blogPosts"":[],""total...",We make custom models for specialized NLP appl...,23,"[{'user': 'Mohamedelamoury', 'fullname': 'EL A...",0,[]


In [17]:
json.loads(df["user_meta"][0])

{'lastUserActivities': [],
 'blogPosts': [],
 'totalBlogPosts': 0,
 'canReadDatabase': False,
 'canManageEntities': False,
 'canReadEntities': False,
 'canImpersonate': False,
 'canManageBilling': False,
 'communityScore': 0,
 'collections': [],
 'datasets': [],
 'models': [{'author': 'ricardotalavera',
   'authorData': {'_id': '642db9df1133cabbf82fc8e5',
    'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/noauth/SjK6nwIKwPgToCmWn3AGC.jpeg',
    'fullname': 'Ricardo Talavera',
    'name': 'ricardotalavera',
    'type': 'user',
    'isPro': False,
    'isHf': False,
    'isHfAdmin': False,
    'isMod': False},
   'downloads': 10,
   'gated': False,
   'id': 'ricardotalavera/aak-bert-base-cased-cpc-ricardo-talavera',
   'availableInferenceProviders': [],
   'lastModified': '2023-04-07T15:47:50.000Z',
   'likes': 0,
   'pipeline_tag': 'text-classification',
   'private': False,
   'repoType': 'model',
   'isLikedByUser': False,
   'widgetOutputUrls': []},
  {'author

In [18]:
# Models created by users
df["models"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["models"] if x else {})
df["models_count"] = df["models"].parallel_apply(lambda x: len(x))

# Datasets created by users
df["datasets"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["datasets"] if x else {})
df["datasets_count"] = df["datasets"].parallel_apply(lambda x: len(x))

# User information
df["user_avatar"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["u"]["avatarUrl"] if x else None)
df["user_pro"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["u"]["isPro"] if x else None)

# User organizations
df["orgs"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["u"]["orgs"] if x else None)

# User languages
df["accept_languages"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["acceptLanguages"] if x else None)
df.to_csv("../data/process/user.info.tsv", index=False, sep="\t")
df.head(5)

,user_id,user_name,user_meta,team,follower_amount,follower_meta,following_amount,following_meta,models,models_count,datasets,datasets_count,user_avatar,user_pro,orgs,accept_languages
0,ricardotalavera,Ricardo Talavera,"{""lastUserActivities"":[],""blogPosts"":[],""total...",None yet,0,[],0,[],"[{'author': 'ricardotalavera', 'authorData': {...",4,[],0,https://cdn-avatars.huggingface.co/v1/producti...,False,[],[en]
1,Teetouch,Teetouch Jaknamon,"{""lastUserActivities"":[],""blogPosts"":[],""total...",None,0,[],0,[],"[{'author': 'Teetouch', 'authorData': {'_id': ...",1,[],0,/avatars/d150cef7965877a88d7400c431c626d7.svg,False,[],[en]
2,crossdelenna,Cross,"{""lastUserActivities"":[{""time"":""2025-02-28T19:...",\n\t,0,[],1,[],"[{'author': 'crossdelenna', 'authorData': {'_i...",9,"[{'author': 'crossdelenna', 'downloads': 153, ...",5,/avatars/3fa94f159902cfaeed150ebf076d4c0a.svg,False,[],[en]
3,orestxherija,Orest Xherija,"{""lastUserActivities"":[],""blogPosts"":[],""total...",None yet,0,[],0,[],"[{'author': 'orestxherija', 'authorData': {'_i...",2,[],0,/avatars/4dcad2bb7c32b98a801fe7380116e947.svg,False,[],[en]
4,nlptown,NLP Town,"{""lastUserActivities"":[],""blogPosts"":[],""total...",We make custom models for specialized NLP appl...,23,"[{'user': 'Mohamedelamoury', 'fullname': 'EL A...",0,[],"[{'author': 'nlptown', 'authorData': {'_id': '...",2,[],0,https://cdn-avatars.huggingface.co/v1/producti...,False,[],[en]
